In [48]:
## Load Packages
import pandas as pd
import re
import numpy as np
from geopy.geocoders import GoogleV3
from geopy.extra.rate_limiter import RateLimiter
from google.cloud import bigquery
from oauth2client.service_account import ServiceAccountCredentials
import os

In [49]:
#Setup Google API Key
api_key = ""
geolocator = GoogleV3(api_key=api_key)

In [50]:
#Setup BigQuery Credentials
gbq_credentials = ''

#Apply credentials to BigQuery Client
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = gbq_credentials
client = bigquery.Client()
dataset_id = "ppl-stage-bigquery.pa_people_derived_tables"
dataset = bigquery.Dataset(dataset_id)
dataset.location = ""
try:
    dataset = client.create_dataset(dataset)  # API request
    print("Created dataset {}.{}".format(client.project, dataset.dataset_id))
except Exception as e:
    print(e)

409 POST https://bigquery.googleapis.com/bigquery/v2/projects/ppl-stage-bigquery/datasets?prettyPrint=false: Already Exists: Dataset ppl-stage-bigquery:pa_people_derived_tables


In [51]:
#SQL Query and project
bigquery_project = 'ppl-stage-bigquery'
query = """WITH parsed_education AS 
            ( SELECT 
                opportunity_id,
                degree,
                school_start_date,
                school_end_date,
                university,
                field
            FROM (
                SELECT 
                    lo.id AS opportunity_id,
                    los.degree AS degree,
                    PARSE_DATE('%m/%d/%Y', CONCAT(CAST(los.start_month AS STRING), '/1/', CAST(los.start_year AS STRING))) AS school_start_date,
                    PARSE_DATE('%m/%d/%Y', CONCAT(CAST(los.end_month AS STRING), '/1/', CAST(los.end_year AS STRING))) AS school_end_date,
                    los.org AS university,
                    los.field AS field,
                    ROW_NUMBER() OVER (
                        PARTITION BY lo.id 
                        ORDER BY PARSE_DATE('%m/%d/%Y', CONCAT(CAST(los.end_month AS STRING), '/1/', CAST(los.end_year AS STRING))) DESC, 
                                lo.id DESC
                    ) AS school_rank
                FROM lever.opportunity_school los
                LEFT JOIN lever.resume lr ON los.resume_id = lr.id
                LEFT JOIN lever.opportunity lo ON lr.opportunity_id = lo.id
            ) ranked
            WHERE school_rank = 1
            ORDER BY opportunity_id DESC
            ),
            parsed_background AS 
            (
            SELECT
                lop.resume_id                                                                                          AS resume_id
                , lo.id                                                                                                  AS opportunity_id
                , PARSE_DATE('%m/%d/%Y', CONCAT(CAST(lop.start_month AS STRING), '/1/', CAST(lop.start_year AS STRING))) AS previous_company_start_date
                , PARSE_DATE('%m/%d/%Y', CONCAT(CAST(lop.end_month AS STRING), '/1/', CAST(lop.end_year AS STRING)))     AS previous_company_end_date
                , lop.org                                                                                                AS previous_company
                , INITCAP(REGEXP_REPLACE(lop.org, r'^(.*?)[/(;:,].*$', r'\1'))                                             AS formatted_company
                , lop.title                                                                                              AS previous_title
                , ROW_NUMBER() OVER (
                    PARTITION BY lo.id
                    ORDER BY 
                        PARSE_DATE('%m/%d/%Y', CONCAT(CAST(lop.start_month AS STRING), '/1/', CAST(lop.start_year AS STRING))) 
                        DESC, lo.id DESC
                                )                                                                                        AS rank
                FROM 
                lever.opportunity_position lop
                LEFT JOIN 
                lever.resume lr
                ON lop.resume_id = lr.id
                LEFT JOIN 
                lever.opportunity lo
                ON lr.opportunity_id = lo.id
            )
            SELECT 
                pb.opportunity_id
                , pb.previous_company_start_date
                , pb.previous_company_end_date
                , DATE_DIFF(pb.previous_company_end_date, pb.previous_company_start_date, MONTH) AS tenure_months
                , pb.previous_company
                , pb.formatted_company
                , pb.previous_title
                , pe.degree
                , pe.school_start_date
                , pe.school_end_date
                , pe.university
                , pe.field
                , pb.rank 
                FROM parsed_background pb
                LEFT JOIN 
                    parsed_education pe
                    ON pb.opportunity_id = pe.opportunity_id
                ORDER BY pb.opportunity_id, rank ASC"""

In [52]:
#specify the project and result
query_job = client.query(query, project=bigquery_project)
results = query_job.result()

#print results back to GBQ
df = (
    client.query(query, project=bigquery_project, location='europe-west2')
    .result()
    .to_dataframe()
)

In [53]:
# Transform raw university text to a clean format
def clean_university(university):
    if university is None:
        return None
    if ',' not in university and '-' not in university and ';' not in university and ':' not in university and '(' not in university:
        return university.title()
    tokens = [token.strip() for token in re.split(r'[\,\-;/:|\(\)]', university) if token]
    name = university 
    for token in tokens:
        lower_token = token.lower()
        if 'universi' in lower_token or 'college' in lower_token or 'institut' in lower_token or 'school' in lower_token:
            name = token
            break
    return name.title()

df['parsed_university'] = df['university'].apply(clean_university)

In [54]:
# Define the new schema
new_schema = [
    bigquery.SchemaField("opportunity_id", "STRING"),
    bigquery.SchemaField("previous_company_start_date", "DATE"),
    bigquery.SchemaField("previous_company_end_date", "DATE"),
    bigquery.SchemaField("tenure_months", "INTEGER"),
    bigquery.SchemaField("previous_company", "STRING"),
    bigquery.SchemaField("formatted_company", "STRING"),
    bigquery.SchemaField("previous_title", "STRING"),
    bigquery.SchemaField("degree", "STRING"),
    bigquery.SchemaField("school_start_date", "DATE"),
    bigquery.SchemaField("school_end_date", "DATE"),
    bigquery.SchemaField("university", "STRING"),
    bigquery.SchemaField("field", "STRING"),
    bigquery.SchemaField("rank", "INTEGER")
    # Add or modify fields as necessary
]

#create a derived destination table
destination_table = f"{client.project}.pa_people_derived_tables.parsed_backgrounds"

# Set up the job configuration with the new schema
job_config = bigquery.LoadJobConfig(
    schema=new_schema,
    schema_update_options=[
        bigquery.SchemaUpdateOption.ALLOW_FIELD_ADDITION
    ]
)

# Load the DataFrame into BigQuery using the defined schema
job = client.load_table_from_dataframe(df, destination_table, job_config=job_config)
job.result()  # Wait for the job to complete

LoadJob<project=ppl-stage-bigquery, location=europe-west2, id=6943831c-feed-4b00-baf9-d01b6912d5bb>